In [9]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np
import tensorflow as tf
from typing import Tuple

In [10]:
!wget -Nq https://raw.githubusercontent.com/MicrosoftDocs/tensorflow-learning-path/main/intro-keras/kintro.py
from kintro import *

In [16]:
learning_rate = 0.1
batch_size = 64

model = NeuralNetwork()

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate)
metrics = ['accuracy']
model.compile(optimizer, loss_fn, metrics)

In [17]:
epochs = 5
print('\nFitting:')
model.fit(train_dataset, epochs=epochs)


Fitting:
Epoch 1/5
938/938 [==============================] - 2s 1ms/step - loss: 0.6237 - accuracy: 0.7803
Epoch 2/5
938/938 [==============================] - 2s 1ms/step - loss: 0.4543 - accuracy: 0.8368
Epoch 3/5
938/938 [==============================] - 2s 2ms/step - loss: 0.4176 - accuracy: 0.8510
Epoch 4/5
938/938 [==============================] - 2s 1ms/step - loss: 0.3956 - accuracy: 0.8580
Epoch 5/5
938/938 [==============================] - 2s 1ms/step - loss: 0.3807 - accuracy: 0.8627


In [18]:
print('\nEvaluating:')
(test_loss, test_accuracy) = model.evaluate(test_dataset)
print(f'\nTest accuracy: {test_accuracy * 100:>0.1f}%, test loss: {test_loss:>8f}')


Evaluating:
157/157 [==============================] - 1s 1ms/step - loss: 0.4175 - accuracy: 0.8515

Test accuracy: 85.1%, test loss: 0.417497


In [19]:
model.save('outputs/model')

INFO:tensorflow:Assets written to: outputs/model/assets


INFO:tensorflow:Assets written to: outputs/model/assets
